In [ ]:
import torch
import torch.nn as nn
from skimage import io, transform

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self,image_path,targets,augmentations=None):
        self.image_path = image_path
#         self.features = features
        self.targets = targets
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_path)
    
    def __getitem__(self,item):
        image = io.imread(self.image_path[item])
#         features = self.features[item,:]
        targets = self.targets[item]
        
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
            
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        return {
            "image": torch.tensor(image, dtype=torch.float),
#             "features": torch.tensor(features, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.float),
        }
        

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm
######  Thank you NIKITA KOZODOI https://www.kaggle.com/kozodoi/timm-pytorch-image-models for your kaggle dataset

In [ ]:
# timm.list_models('*efficientnet*')

#### 

In [ ]:
import torch
import torch.nn as nn

model_name = 'tf_efficientnet_b2_ns'
file_name = "../input/tf-efficientnet-b2-ns/tf_efficientnet_b2_ns-00306e48.pth"
out_dim    = 1

def get_model():
    model = timm.create_model(model_name, pretrained=False)
    
    model.load_state_dict(torch.load(file_name))
    model.classifier = nn.Linear(model.classifier.in_features, 
                             out_dim)
    return model

In [ ]:
import pandas as pd
import numpy as np
from sklearn import model_selection
df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
df["kfold"] = -1

df = df.sample(frac=1).reset_index(drop=True)

kf = model_selection.StratifiedKFold(n_splits=5, shuffle=False)

for fold, (train_idx, val_idx) in enumerate(kf.split(X=df,y=df.Pawpularity.values)):
    print(len(train_idx), len(val_idx))
    df.loc[val_idx, 'kfold'] = fold

In [ ]:
import os
from sklearn import metrics
import albumentations
device = 'cuda'
epochs = 12
data_path = '../input/petfinder-pawpularity-score'
train_aug = albumentations.Compose(                  ##  AUGMENTATIONs TAKEN FROM ABHISHEK THAKUR's tez Pawpular training
    [
        albumentations.Resize(256,256, p=1),
        albumentations.HueSaturationValue(
            hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5
        ),
        albumentations.RandomBrightnessContrast(
            brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5
        ),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

valid_aug = albumentations.Compose(
    [
        albumentations.Resize(256, 256, p=1),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)


In [ ]:
scores = []
for fold in range(5):
        model = get_model()
        model.to(device)
        df_train = df[df.kfold != fold].reset_index(drop=True)
        df_valid = df[df.kfold == fold].reset_index(drop=True)

        df_train = df_train.drop(columns = 'kfold')
        df_valid = df_valid.drop(columns = 'kfold')

        train_images = df_train.Id.values.tolist()
        train_images = [os.path.join(data_path,'train',i + '.jpg') for i in train_images]
        valid_images = df_valid.Id.values.tolist()
        valid_images = [os.path.join(data_path,'train',i + '.jpg') for i in valid_images]

        train_targets = df_train.Pawpularity.values
        valid_targets = df_valid.Pawpularity.values

        train_dataset = CustomDataset(image_path = train_images,targets = train_targets,augmentations=train_aug)
        train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=64,shuffle=True,pin_memory=True) 
        valid_dataset = CustomDataset(image_path = valid_images,targets =valid_targets,augmentations=valid_aug)
        valid_loader = torch.utils.data.DataLoader(valid_dataset,batch_size=64,shuffle=False,pin_memory=True) 

        optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)
        
        for epoch in range(epochs):
            
            model.train()
            for data in train_loader:
                inputs = data['image']
                targets = data['targets']

                inputs = inputs.to(device, dtype=torch.float)
                targets = targets.to(device, dtype=torch.float)

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = nn.MSELoss()(outputs, targets.view(-1, 1))
                loss.backward()
                optimizer.step()
           
            model.eval()
            final_targets = []
            final_outputs = []
                
            with torch.no_grad():
                for data in valid_loader:
                    inputs = data['image']
                    targets = data['targets']
                    inputs = inputs.to(device, dtype=torch.float)
                    targets = targets.to(device, dtype=torch.float)

                    output = model(inputs)

                    targets = targets.detach().cpu().numpy().tolist()
                    output = output.detach().cpu().numpy().tolist()
                    final_outputs.extend(output)
                    final_targets.extend(targets)
                    RMSE = np.sqrt(metrics.mean_squared_error(final_targets,final_outputs))
            
            print(f'you are in fold {fold} and  Epoch = {epoch}   valid RMSE={RMSE}')
        torch.save(model.state_dict(),'model-epoch'+str(fold)+'.pth')
        scores.append(RMSE)
            

In [ ]:
from itertools import chain
max = 200
model_no = 0
for i in range(5):
    if score[i] < max:
        model_no = i
        max = scores[i]
        
        
model_f = get_model()
model_f.to(device)
model_f.load_state_dict(torch.load('./model-epoch'+str(model_no)+'.pth'))
data_path = '../input/petfinder-pawpularity-score'
device = 'cuda'
df_test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test_images = df_test.Id.values.tolist()
test_images = [os.path.join(data_path,'test',i + '.jpg') for i in test_images]

test_dataset =  CustomDataset(image_path = test_images,targets = np.ones(len(test_images)),augmentations=valid_aug)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=64,shuffle=False) 


final_outputs = []
 
with torch.no_grad():
    for data in test_loader:
        inputs = data['image']
        inputs = inputs.to(device, dtype=torch.float)
        output = model_f(inputs)
        output = output.detach().cpu().numpy().tolist()
        final_outputs.extend(output)
        

final_outputs = list(chain.from_iterable(final_outputs))        
submission = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
submission['Pawpularity'] = final_outputs
submission.to_csv('submission.csv',index = False)